# Import Tools

In [1]:
# !pip install ultralytics

In [3]:
# !python --version

In [1]:
import numpy as np
import torch
import cv2
import os
torch.__version__

'2.2.1+cu121'

In [2]:
import ultralytics
from ultralytics import YOLO
ultralytics.__version__

C:\Users\USER\anaconda3\envs\groundingdino\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'8.1.24'

In [3]:
torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No cuda"

'NVIDIA GeForce RTX 4060'

# Predict with SAM

## Image

In [15]:
from ultralytics import SAM

# sam_l.pt and sam_b.pt

model = SAM('sam_b.pt')

annotations = model.predict('images/10.jpg', show=True)


image 1/1 C:\Users\USER\Downloads\Auto-Annotation-Using-YOLOv8-and-SAm\images\10.jpg: 1024x1024 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 1 10, 1 11, 1 12, 1 13, 1 14, 1 15, 1 16, 1 17, 1 18, 1 19, 1 20, 1 21, 1 22, 1 23, 1 24, 1 25, 1 26, 1 27, 1 28, 1 29, 1 30, 1 31, 1 32, 1 33, 1 34, 1 35, 1 36, 1 37, 1 38, 1 39, 1 40, 1 41, 1 42, 1 43, 1 44, 1 45, 1 46, 1 47, 1 48, 1 49, 1 50, 1 51, 1 52, 1 53, 1 54, 1 55, 1 56, 1 57, 1 58, 1 59, 1 60, 1 61, 1 62, 1 63, 1 64, 1 65, 1 66, 1 67, 1 68, 1 69, 1 70, 1 71, 1 72, 1 73, 1 74, 1 75, 1 76, 1 77, 1 78, 1 79, 1 80, 1 81, 1 82, 1 83, 1 84, 1 85, 1 86, 1 87, 1 88, 1 89, 1 90, 1 91, 1 92, 1 93, 1 94, 1 95, 1 96, 1 97, 1 98, 1 99, 1 100, 1 101, 1 102, 1 103, 1 104, 1 105, 1 106, 1 107, 6763.1ms
Speed: 5.0ms preprocess, 6763.1ms inference, 4.0ms postprocess per image at shape (1, 3, 1024, 1024)


In [5]:
# annotations[0].boxes

## Video

In [12]:
# results = model.predict('videos/giphy.gif', device = 'gpu', show = True, save=True)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/70) C:\Users\USER\Downloads\Auto-Annotation-Using-YOLOv8-and-SAm\videos\giphy.gif: 1024x1024 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 1 10, 1 11, 1 12, 1 13, 1 14, 1 15, 1 16, 1 17, 1 18, 1 19, 1 20, 1 21, 1 22, 1 23, 1 24, 1 25, 1 26, 1 27, 1 28, 1 29, 1 30, 1 31, 1 32, 1 33, 1 34, 1 35, 1 36, 1 37, 1 38, 1 39, 1 40, 1 41, 1 42, 1 43, 1 44, 1 45, 1 46, 1 47, 1 48, 1 49, 1 50, 1 51, 1 52, 1 53, 1 54, 1 55, 1 56, 6875.2ms
video 1/1

## Webcam

In [9]:
# results = model.predict(0, device = 0, show = True)

# Generate Segmentation Dataset Using a Detection Model

In [19]:
from ultralytics.data.annotator import auto_annotate

auto_annotate(data="images", det_model="yolov8x.pt", sam_model='sam_l.pt')


image 1/3 C:\Users\USER\Downloads\Auto-Annotation-Using-YOLOv8-and-SAm\images\10.jpg: 384x640 3 persons, 1 backpack, 4 handbags, 21.0ms
image 2/3 C:\Users\USER\Downloads\Auto-Annotation-Using-YOLOv8-and-SAm\images\8.png: 448x640 7 cars, 1 truck, 32.0ms
image 3/3 C:\Users\USER\Downloads\Auto-Annotation-Using-YOLOv8-and-SAm\images\9.jpg: 384x640 16 persons, 1 chair, 1 tv, 44.0ms
Speed: 1.7ms preprocess, 32.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)


In [ ]:
def visualize_annotations(image_dir, annotation_dir):
    for filename in os.listdir(image_dir):
        if filename.endswith(".jpg") or filename.lower().endswith(".png") or filename.endswith(".jpeg"):
            image_path = os.path.join(image_dir, filename)
            image = cv2.imread(image_path)

            annot_path = os.path.join(annotation_dir, filename.split(".")[0] + ".txt")
            if os.path.exists(annot_path):
                with open(annot_path, "r") as txt_file:
                    for line in txt_file:
                        data = list(map(float, line.split()))
                        class_id = data[0]
                        # Polygon corrdinates
                        points = np.array(data[1:]).reshape(-1, 2) * np.array([image.shape[1], image.shape[0]])
                        points = points.astype(np.int32)

                        # Draw the polygon on the image
                        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                        # Draw the label on the image
                        cv2.putText(image, f'Class {class_id}', tuple(points[0]), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                # Display the image with annotations
                cv2.imshow('Annotated Image', image)
                cv2.waitKey(0)
                cv2.destroyAllWindows()


visualize_annotations("images", "images_auto_annotate_labels")

## Try with Video

In [12]:
# cap = cv2.VideoCapture('path/to/your/video.mp4')